## Harmony Py Library
### Getting data using AWS credentials, S3 URLs, and downloading with AWS boto3

In [ ]:
import datetime as dt
import sys
sys.path.append('..')
from harmony import BBox, Client, Collection, LinkType, Request, s3_components

In [ ]:
request = Request(
    collection=Collection(id='C1234088182-EEDTEST'),
    spatial=BBox(-165, 52, -140, 77),
    temporal={
        'start': dt.datetime(2010, 1, 1),
        'stop': dt.datetime(2020, 12, 30)
    },
    variables=['blue_var'],
    max_results=2,
    crs='EPSG:3995',
    format='image/tiff',
    height=512,
    width=512
)

request.is_valid()

Cloud access credentials can be retrieved from an instantiated Client.

In [ ]:
harmony_client = Client()  # assumes .netrc usage
job_id = harmony_client.submit(request)

Now let's get the data URLs from Harmony, but request S3 URLs instead of the default HTTPS. We also request temporary AWS credentials that we can use to authenticate and download the data.

In [ ]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
print(results)
creds = harmony_client.aws_credentials()

Now we'll use the AWS Python library boto3 to download our results from the S3 bucket, providing boto3 with our temporary credentials that Harmony supplied:

In [ ]:
#
# NOTE: Execution of this cell will only succeed within the AWS us-west-2 region. 
#

import boto3

s3 = boto3.client('s3', **creds)
for url in results:
    bucket, obj, fn = s3_components(url)
    with open(fn, 'wb') as f:
        s3.download_fileobj(bucket, obj, f)